In [55]:
import pandas as pd
import re

# Pré-processamento dos Dados

### Carregar Base de dados

In [56]:
laptop_price = pd.read_csv("../datasets/raw/laptop_price.csv", encoding="ISO-8859-1")
laptop_price.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


### Verificar tipo das colunas e dropar coluna laptop_id

In [57]:
laptop_price = laptop_price.rename(columns={'ScreenResolution': 'screen_resolution'})
laptop_price.columns = laptop_price.columns.str.lower()
laptop_price = laptop_price.drop(columns='laptop_id')
laptop_price.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   company            1303 non-null   object 
 1   product            1303 non-null   object 
 2   typename           1303 non-null   object 
 3   inches             1303 non-null   float64
 4   screen_resolution  1303 non-null   object 
 5   cpu                1303 non-null   object 
 6   ram                1303 non-null   object 
 7   memory             1303 non-null   object 
 8   gpu                1303 non-null   object 
 9   opsys              1303 non-null   object 
 10  weight             1303 non-null   object 
 11  price_euros        1303 non-null   float64
dtypes: float64(2), object(10)
memory usage: 904.4 KB


### Verificar quantidade de valores únicos

In [58]:
print("Quantidade de Valores únicos por coluna")
for column in laptop_price.columns:
    print(f"{column}: {len(laptop_price[column].unique())}")

Quantidade de Valores únicos por coluna
company: 19
product: 618
typename: 6
inches: 18
screen_resolution: 40
cpu: 118
ram: 9
memory: 39
gpu: 110
opsys: 9
weight: 179
price_euros: 791


## Inches

In [59]:
def tratative_inches(row):
    if row < 14:
        row = '14-'
    elif row > 16:
        row = '16+'
    else:
        row = '14~16'
        
    return row
        
laptop_price.inches = laptop_price.inches.apply(tratative_inches)

### Obter fabricante da cpu e gpu

In [60]:
laptop_price['cpu_manufacturer'] = laptop_price.cpu.str.split(n=1, expand=True)[0]
laptop_price['gpu_manufacturer'] = laptop_price.gpu.str.split(n=1, expand=True)[0]

### Obter Ghz da Cpu

In [61]:
laptop_price['cpu_ghz'] = laptop_price.cpu.str.rsplit(n=1, expand=True)[1]
laptop_price.cpu_ghz = laptop_price.cpu_ghz.str.lower().replace(to_replace='ghz', value='', regex=True)

### Tratar memory

In [62]:
laptop_price.memory.replace(to_replace = '([.][0])|([G][B])', value = '', regex=True, inplace=True)
laptop_price.memory.replace(to_replace = '([T][B])', value = '000', regex=True, inplace=True)
laptop_price.memory.replace(to_replace = '( )+', value = ' ', regex=True, inplace=True)

laptop_price['memory_principal_gb'] = 0
laptop_price['memory_principal_type'] = 'empty'
laptop_price['memory_secondary_gb'] = 0
laptop_price['memory_secondary_type'] = 'empty'

### Verificar valores únicos

In [63]:
laptop_price.memory.unique()

array(['128 SSD', '128 Flash Storage', '256 SSD', '512 SSD', '500 HDD',
       '256 Flash Storage', '1000 HDD', '32 Flash Storage',
       '128 SSD + 1000 HDD', '256 SSD + 256 SSD', '64 Flash Storage',
       '256 SSD + 1000 HDD', '256 SSD + 2000 HDD', '32 SSD', '2000 HDD',
       '64 SSD', '1000 Hybrid', '512 SSD + 1000 HDD', '1000 SSD',
       '256 SSD + 500 HDD', '128 SSD + 2000 HDD', '512 SSD + 512 SSD',
       '16 SSD', '16 Flash Storage', '512 SSD + 256 SSD',
       '512 SSD + 2000 HDD', '64 Flash Storage + 1000 HDD', '180 SSD',
       '1000 HDD + 1000 HDD', '32 HDD', '1000 SSD + 1000 HDD',
       '512 Flash Storage', '128 HDD', '240 SSD', '8 SSD', '508 Hybrid',
       '512 SSD + 1000 Hybrid', '256 SSD + 1000 Hybrid'], dtype=object)

In [64]:
for i, row in enumerate(laptop_price['memory']):
    row = row.lower()
    for j, items in enumerate(row.split('+ ')):
        valores = items.split(' ', 1)
        if j == 0:
            memory = 'principal'
        else:
            memory = 'secondary'
        laptop_price.loc[i, f'memory_{memory}_gb'] += eval(valores[0])
        valores = valores[1]
        if valores[-1] == ' ':
            laptop_price.loc[i, f'memory_{memory}_type'] = valores[:-1]
        else:
            laptop_price.loc[i, f'memory_{memory}_type'] = valores

laptop_price.drop(columns=['memory'], inplace=True)

### SO

In [65]:
laptop_price.opsys.unique()

array(['macOS', 'No OS', 'Windows 10', 'Mac OS X', 'Linux', 'Android',
       'Windows 10 S', 'Chrome OS', 'Windows 7'], dtype=object)

In [66]:
laptop_price.opsys.replace(to_replace ='Mac OS X', value = 'macOS', inplace=True)
laptop_price.opsys.replace(to_replace =['Windows 10 S', 'Windows 10', 'Windows 7'], value = 'Windows', inplace=True)
laptop_price.opsys.replace(to_replace =['No OS', 'Android', 'Chrome OS'], value = 'Outros', inplace=True)

### Remover GB em Ram

In [67]:
laptop_price.ram = laptop_price.ram.str.lower().replace(to_replace = '(gb)', value = '', regex=True)

### Filtrar screen_resolution

In [68]:
laptop_price.screen_resolution = laptop_price.screen_resolution.str.extract(r'([0-9]+[x][0-9]+)')

In [69]:
laptop_price['x_screen'] = laptop_price.screen_resolution.str.split('x', expand=True)[0]
laptop_price['y_screen'] = laptop_price.screen_resolution.str.split('x', expand=True)[1]

In [70]:
laptop_price.screen_resolution = laptop_price.x_screen + 'x' + laptop_price.y_screen

### Filtrar Coluna Weight

In [71]:
laptop_price.weight = laptop_price.weight.str.lower().replace(to_replace = '(kg)', value = '', regex=True)

### Valores máximos e minimos

In [72]:
laptop_price.describe()

,price_euros,memory_principal_gb,memory_secondary_gb
count,1303.000000,1303.000000,1303.000000
mean,1123.686992,442.615503,168.288565
std,699.009043,356.225802,402.618575
min,174.000000,8.000000,0.000000
25%,599.000000,256.000000,0.000000
50%,977.000000,256.000000,0.000000
75%,1487.880000,512.000000,0.000000
max,6099.000000,2000.000000,2000.000000


### Fazer downcasting das colunas

In [73]:
laptop_price.head()

,company,product,typename,inches,screen_resolution,cpu,ram,gpu,opsys,weight,price_euros,cpu_manufacturer,gpu_manufacturer,cpu_ghz,memory_principal_gb,memory_principal_type,memory_secondary_gb,memory_secondary_type,x_screen,y_screen
0,Apple,MacBook Pro,Ultrabook,14-,2560x1600,Intel Core i5 2.3GHz,8,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,Intel,Intel,2.3,128,ssd,0,empty,2560,1600
1,Apple,Macbook Air,Ultrabook,14-,1440x900,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,macOS,1.34,898.94,Intel,Intel,1.8,128,flash storage,0,empty,1440,900
2,HP,250 G6,Notebook,14~16,1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,Outros,1.86,575.00,Intel,Intel,2.5,256,ssd,0,empty,1920,1080
3,Apple,MacBook Pro,Ultrabook,14~16,2880x1800,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,1.83,2537.45,Intel,AMD,2.7,512,ssd,0,empty,2880,1800
4,Apple,MacBook Pro,Ultrabook,14-,2560x1600,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,Intel,Intel,3.1,256,ssd,0,empty,2560,1600


In [74]:
items = {
    'ram': 'ram_gb',
    'weight': 'weight_kg'
}
laptop_price.rename(columns=items, inplace=True)

In [75]:
laptop_price[laptop_price.cpu_manufacturer == 'Samsung']

,company,product,typename,inches,screen_resolution,cpu,ram_gb,gpu,opsys,weight_kg,price_euros,cpu_manufacturer,gpu_manufacturer,cpu_ghz,memory_principal_gb,memory_principal_type,memory_secondary_gb,memory_secondary_type,x_screen,y_screen
1191,Samsung,Chromebook Plus,2 in 1 Convertible,14-,2400x1600,Samsung Cortex A72&A53 2.0GHz,4,ARM Mali T860 MP4,Outros,1.15,659.0,Samsung,ARM,2.0,32,flash storage,0,empty,2400,1600


In [76]:
def downcasting(df, columns, type):
    for column in columns:
        df[column] = df[column].astype(type)
    
laptop_price.drop(columns=['cpu', 'gpu', 'product'], inplace=True)
laptop_price.drop(index=1191, inplace=True)
    
items = {
    'int16': ['ram_gb', 'memory_principal_gb', 'memory_secondary_gb', 'x_screen', 'y_screen'],
    'float32': ['weight_kg', 'cpu_ghz'],
    'float64': ['price_euros'],
    'category': ['company', 'typename', 'screen_resolution', 'opsys', 'cpu_manufacturer', 'gpu_manufacturer', 'memory_principal_type', 'memory_secondary_type', 'inches']
}
    
for tipo, coluna in items.items():
    downcasting(laptop_price, coluna, tipo)    

### Ver Redução de memoria

In [77]:
laptop_price.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1302 entries, 0 to 1302
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   company                1302 non-null   category
 1   typename               1302 non-null   category
 2   inches                 1302 non-null   category
 3   screen_resolution      1302 non-null   category
 4   ram_gb                 1302 non-null   int16   
 5   opsys                  1302 non-null   category
 6   weight_kg              1302 non-null   float32 
 7   price_euros            1302 non-null   float64 
 8   cpu_manufacturer       1302 non-null   category
 9   gpu_manufacturer       1302 non-null   category
 10  cpu_ghz                1302 non-null   float32 
 11  memory_principal_gb    1302 non-null   int16   
 12  memory_principal_type  1302 non-null   category
 13  memory_secondary_gb    1302 non-null   int16   
 14  memory_secondary_type  1302 non-null   c

In [78]:
ordem = ['company',
        'typename', 
        'inches',
        'screen_resolution',
        'x_screen', 
        'y_screen', 
        'cpu_manufacturer', 
        'cpu_ghz',
        'gpu_manufacturer', 
        'ram_gb', 
        'memory_principal_gb',
        'memory_principal_type', 
        'memory_secondary_gb', 
        'memory_secondary_type',
        'opsys',
        'weight_kg',
        'price_euros'
]

laptop_price = laptop_price[ordem]

In [79]:
laptop_price.head()

,company,typename,inches,screen_resolution,x_screen,y_screen,cpu_manufacturer,cpu_ghz,gpu_manufacturer,ram_gb,memory_principal_gb,memory_principal_type,memory_secondary_gb,memory_secondary_type,opsys,weight_kg,price_euros
0,Apple,Ultrabook,14-,2560x1600,2560,1600,Intel,2.3,Intel,8,128,ssd,0,empty,macOS,1.37,1339.69
1,Apple,Ultrabook,14-,1440x900,1440,900,Intel,1.8,Intel,8,128,flash storage,0,empty,macOS,1.34,898.94
2,HP,Notebook,14~16,1920x1080,1920,1080,Intel,2.5,Intel,8,256,ssd,0,empty,Outros,1.86,575.00
3,Apple,Ultrabook,14~16,2880x1800,2880,1800,Intel,2.7,AMD,16,512,ssd,0,empty,macOS,1.83,2537.45
4,Apple,Ultrabook,14-,2560x1600,2560,1600,Intel,3.1,Intel,8,256,ssd,0,empty,macOS,1.37,1803.60


### Salvar para Pickle

In [80]:
laptop_price.to_pickle("../datasets/processed/laptop_price.pkl")
laptop_price.to_excel("../outputs/planilhas/laptop_price.xlsx")